# Astropy: Units and Constants

**Note:** Many packages dealing with physics have a `units`-submodule. Their functionality is very similar and we can use the `astropy`-module as an example. 

## Documentation

For more information about the features presented below, you can read the
[astropy.units](http://docs.astropy.org/en/stable/units/index.html) docs.

## Purpose of the module
We often need to deal with quantities consisting of a numerical value and a *phsyical unit*. Keeping track of concrete units in more complex expressions manually is very prone to errors and cumbersome. As an example consider the following dimensionless expression:
$$
q = \left(\frac{T}{2\pi GM_{\odot}} \right)^{1/3} v.
$$
The `astropy.units`-module allows us to comfortably deal with such expressions.

## Representing units and quantities (numbers with attached units)

### Units

Astropy includes a powerful framework for units which allows us to attach
units to scalars and `numpy`-arrays. These *quantities* can be manipulated and combined at will while `python` keeps track of the units.

Names for units are intuitive and astropy really contains *all* units that we need! Units can be accessed with:

In [ ]:
import astropy.units as au

# basic units
u = au.m  # meter (irreducible base unit)
print(type(u))
print(u.to(au.km)) # conversion to km
print(u.find_equivalent_units())  # list other relevant length units

**Notes:**
- See [this link](http://docs.astropy.org/en/stable/units/standard_units.html) for a list of base-units (not complete!)
- For a given *base-unit* such as meters, all prefixes (*k*=kilo, *m*=milli) etc. can be used
- The `find_equivalent_units`-function does not list **all** available units, only the *most* relevant ones
- Base units can be arbitrarily combined to compound units

In [ ]:
import astropy.units as au
import astropy.units.imperial as aui

aui.enable()  # enable units used in England / the USA
u = au.m
print(u.find_equivalent_units())

cu = au.kg * au.m / au.s**2  # arbitrary combination of base-units
print(type(cu))
print(cu.find_equivalent_units())

cu = au.kg
print(cu.find_equivalent_units())


There is a specific `dimensionless_unscaled`-unit:

In [ ]:
import astropy.units as su

u = au.km / au.m
print(u.to(au.dimensionless_unscaled))

## Conversion of units
Units can be converted between *equivalent units* without problems:

In [ ]:
import astropy.units as su

u = au.km * au.nm * au.M_earth / (au.s * au.year)
u.to(au.J)

Note that we have in astronomy often the understanding of *equivalent units*, although they are not the same physically:

In [ ]:
import astropy.units as au

# mass and energy are equaivalent via Einsteins relation E=m c**2
u = 1.0 * au.kg
print(u.to(au.J, equivalencies=au.mass_energy()))

# Energy and wavelength are equaivalent via E = h * (c/lambda)
e = 1.0 * au.keV
print(e.to(au.nm, equivalencies=au.spectral()))

w = 500.0 * au.nm
print(w.to(au.keV, equivalencies=au.spectral()))

# pixel scales are often considered equivalent to angles:
omegacam_pixscale = au.pixel_scale(0.21 * au.arcsec / au.pixel)
print((60 * au.arcmin).to(au.pixel, omegacam_pixscale))
print((100 * au.pixel).to(au.arcsec, omegacam_pixscale))

## Quantities and `numpy`-arrays

The most useful feature about the units is the ability to attach them to
numerical scalars or arrays, creating ``Quantity`` objects. These can be used naturally with many `numpy`-functions.

In [ ]:
%matplotlib inline
import numpy as np
import astropy.units as au
import matplotlib.pyplot as plt

q = 3. * au.deg  # scalar quantity
print(type(q))
# a quantity has a value and a unit!
print(q.value, q.unit)

q = 3.0 * au.cm / (2.0 * au.m)
# automatic conversion to dimensionless units
print(np.exp(q), np.exp(3.0e-2 / 2. ))

# numpy-array of 'angles'
a = np.linspace(0.0, 360.0, 40) * au.deg  
b = np.sin(a) # correct interpretation of the units!
print(b)

# plots behave as expected; the values are plotted:
plt.plot(a, b)

### Combining and converting quantities

Quantities can be combined in a very natural way

In [ ]:
import numpy as np
import astropy.units as au

q1 = 3.0 * au.m
q2 = 3.0 * au.cm

q3 = q1 / q2
print(q3.value, q3.unit, q3.to(au.dimensionless_unscaled))

q3 = q1 * q2
print(q3.value, q3.unit, q3.to(au.m**2))

q1 = np.linspace(1.0, 60.0, 10) * au.arcmin
q2 = np.linspace(1.0, 600.0, 10) * au.arcsec

q3 = q1 * q2
print(q3.to(au.deg**2))

Quantities can be converted to the SI or CGS system

In [ ]:
q = (3. * au.cm * au.pc / au.g / au.year**2)
si = q.decompose()  # SI is the default
print(si)

cgs = q.decompose(au.cgs.bases)
print(cgs)

### Physical constants

The [astropy.constants](http://docs.astropy.org/en/stable/constants/index.html) module contains
physical constants relevant for Astronomy, and these are defined with units
attached to them using the ``astropy.units`` framework.

The full list of available physical constants is shown [here](http://docs.astropy.org/en/stable/constants/index.html#module-astropy.constants).

In [ ]:
# We want to calculate the Gravitational force felt by a 100. * u.kg 
# space probe by the Sun, at a distance of 3.2au

import astropy.units as au
import astropy.constants as ac

# Here we use the constant G:
F = (ac.G * 1.0 * au.M_sun * 100. * au.kg) / (3.2 * au.au)**2
print(F.to(au.N))

In [ ]:
ac.G

In [ ]:
import astropy.units as au
import astropy.constants as ac

# An example from the 4th Python-project. We want to calculate 'q' in units of the jupyter mass

radial_vel = 55. * au.m / au.s
period = 4.23 * au.day

q_sun = (period / (2. * np.pi * ac.G * ac.M_sun))**(1/3) * radial_vel
q_jupy = q_sun * ac.M_sun / ac.M_jup

print(q_sun.to(au.dimensionless_unscaled))
print(q_jupy.to(au.dimensionless_unscaled))